## Imports


In [ ]:
!nvidia-smi

Sun Apr 10 14:17:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -Uqq datasets transformers[sentencepiece]
!pip install -Uqq accelerate
!apt install -Uqq git-lfs
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl


E: Command line option 'U' [from -Uqq] is not understood in combination with the other options.


In [ ]:
%reload_ext autoreload
# %autoreload 2
%matplotlib inline

%env WANDB_PROJECT=NLMUT_CLF
%env WANDB_LOG_MODEL=true
%env TOKENIZERS_PARALLELISM = true

env: WANDB_PROJECT=NLMUT_CLF
env: WANDB_LOG_MODEL=true
env: TOKENIZERS_PARALLELISM=true


In [ ]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login 

In [ ]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!pip install -Uqq wandb

import wandb
wandb.login()

wandb: Currently logged in as: cohlem (use `wandb login --relogin` to force relogin)


True

# Dataset


## 16NepaliNews


In [ ]:
!wget --header "Host: raw.githubusercontent.com" --user-agent "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:98.0) Gecko/20100101 Firefox/98.0" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8" --header "Accept-Language: en-US,en;q=0.5" --referer "https://github.com/sndsabin/Nepali-News-Classifier/blob/master/16NepaliNews.rar" --header "Upgrade-Insecure-Requests: 1" --header "Sec-Fetch-Dest: document" --header "Sec-Fetch-Mode: navigate" --header "Sec-Fetch-Site: cross-site" --header "Sec-Fetch-User: ?1" "https://raw.githubusercontent.com/sndsabin/Nepali-News-Classifier/master/16NepaliNews.rar" --output-document "16NepaliNews.rar"

In [ ]:
# !rm -rf 16NepaliNews/
!unrar x 16NepaliNews.rar

In [ ]:
from pathlib import Path
# !mkdir ./16NepaliNews/16719
path = Path('./16NepaliNews/16719')
path

## Nepali News Dataset Large


In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
! kaggle datasets download -d ashokpant/nepali-news-dataset-large

In [ ]:
ls

In [ ]:
!unzip nepali-news-dataset-large.zip

In [ ]:
ls nepali_news_dataset_20_categories_large/nepali_news_dataset_20_categories_large/

In [ ]:
path = Path('./nepali_news_dataset_20_categories_large/nepali_news_dataset_20_categories_large/')
path.ls()

# Data Cleaning


In [ ]:
# from sklearn import datasets
from sklearn.datasets import load_files

In [ ]:
dataset = load_files(path/'train/', encoding="utf-8", decode_error="ignore", shuffle=False)
dataset_valid = load_files(path/'test/', encoding="utf-8", decode_error="ignore", shuffle=False)
dir(dataset)

In [ ]:
label_map = dict()
for i,name in enumerate(dataset.target_names):
  label_map[i] = name
label_map

In [ ]:
# Train set
df = pd.DataFrame(data=dataset.data, columns=["text"])
df['target'] = pd.Series(dataset.target)
# df['target'] = df['target'].map(label_map)  # Uncomment to use Strings as labels
# df['is_valid'] = False
df.head()

In [ ]:
# Test set
df_test = pd.DataFrame(data=dataset_valid.data, columns=["text"])
df_test['target'] = pd.Series(dataset_valid.target)
# df_test['target'] = df_test['target'].map(label_map)  # Uncomment to use Strings as labels
# df_test['is_valid'] = True
df_test.head()

In [ ]:
# df_full = pd.concat([df, df_test])
# df_full.size

## EDA


In [ ]:
df.head()

In [ ]:
print(df.shape)

In [ ]:
df.target.describe()

In [ ]:
length = df['text'].str.split().str.len().mean()
print(f"Average number of words in 'text' column: {length}")


length = df_test['text'].str.split().str.len().mean()
print(f"Average number of words in 'text' column: {length}")


Data Cleaning goes here


In [ ]:
import re
def clean_text(text):
  # text = re.sub('[^a-z\s]', ' ', text.lower())
  # text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
  text = re.sub('@\w+', ' ', text)
  text = ' '.join(text.split())
  text.replace('\n\n','')
  text.replace('\n',' ')
  text.replace('\t',' ')
  text.replace('\r','')
  return text

df['text'] = df['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)

In [ ]:
length = df['text'].str.split().str.len().mean()
print(f"Average number of words in 'text' column: {length}")

length = df['text'].str.split().str.len().max()
print(f"Maximum number of words in 'text' column: {length}")

length = df['text'].str.split().str.len().min()
print(f"Minimum number of words in 'text' column: {length}")

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
df['target'].value_counts().plot(kind='bar')
plt.axis('off')
plt.show()

In [ ]:
# Number of labels before data cleaning
num_labels = df.target.nunique()
num_labels

In [ ]:
# df['category'].value_counts()[117]

In [ ]:
# Cheap undersampling

# categories = df['category'].value_counts().index
# for i in range(10):
#   category = categories[i]
#   num_to_remove = 12000-i*1200
#   to_remove = np.random.choice(df[df['category']==category].index, size=num_to_remove, replace=False)
#   df.drop(to_remove, inplace=True)

# categories = df['category'].value_counts().index
# value_counts = df['category'].value_counts()
# for i in range(117):
#   category = categories[i]
#   num_to_remove = value_counts[i]- 500   # keep only 1200 data for each class having about 1200 data
#   to_remove = np.random.choice(df[df['category']==category].index, size=num_to_remove, replace=False)
#   df.drop(to_remove, inplace=True)

In [ ]:
# df['target'].value_counts().plot(kind='bar')
# plt.axis('off')
# plt.show()

In [ ]:
df.dropna(subset=['text','target'], how='any', inplace=True)
print(df.shape)

In [ ]:
_ = df[df.text.str.split().str.len() <= 12]
_

In [ ]:
# Delete sentences with less than 4 words
df = df[df.text.str.split().str.len() >= 4]
df.shape

In [ ]:
# Make sure no labels are deleted
assert(num_labels == df.target.nunique())

In [ ]:
df.to_csv(path_or_buf='16NepaliNews_train.csv', index=False)
df_test.to_csv(path_or_buf='16NepaliNews_test.csv', index=False)

!cp 16NepaliNews_train.csv drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_train.csv
!cp 16NepaliNews_test.csv drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_test.csv

# !cp drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_train.csv 16NepaliNews_train.csv 
# !cp drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_test.csv 16NepaliNews_test.csv 

# df = pd.read_csv('16NepaliNews_train.csv')
# df_test = pd.read_csv('16NepaliNews_test.csv')

## Load dataset for New Session


In [ ]:
!cp drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_train.csv 16NepaliNews_train.csv 
!cp drive/MyDrive/Colab\ Notebooks/NLMUT/16NepaliNews_test.csv 16NepaliNews_test.csv 

df = pd.read_csv('16NepaliNews_train.csv')
df_test = pd.read_csv('16NepaliNews_test.csv')

# Transformers


## Transformer Dataset (Define Data Loader)


In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': '16NepaliNews_train.csv', 'test': '16NepaliNews_test.csv' })
dataset

In [ ]:
dataset = dataset.rename_column("target", "labels") # This is needed

In [ ]:
dataset['train'][0]
# Note the '\ufeff' is removed by tokenizer

### Export Dataset


In [ ]:
from huggingface_hub import notebook_login 
notebook_login()

In [ ]:
dataset.push_to_hub("Sakonii/16NepaliNews", private=True)

## Load Dataset from the HUB


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset
dataset = load_dataset('Sakonii/16NepaliNews', streaming=False, use_auth_token=True)

Using custom data configuration Sakonii--16NepaliNews-b9065bf14077793a
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/Sakonii--16NepaliNews-b9065bf14077793a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

## Dataset Tokenization (Changes for each model)


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Rajan/NepaliBERT")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/Rajan/NepaliBERT/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1246361fd733d7f81601e855a3204506449f8b226745d66f58d4f84eaf201c41.36e4050a7f8f6f24536203f25cff98ef0c38bcd695bb9d1efbf39e3b09b9f273
Model config BertConfig {
  "_name_or_path": "Rajan/NepaliBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,

Downloading:   0%|          | 0.00/964k [00:00<?, ?B/s]

storing https://huggingface.co/Rajan/NepaliBERT/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/a3ba5495a5249c5164e69fe430d3d126ad5d6427a55ee70274a812964bbc34f9.2f6fd7c2709e1ba46b4621a2d5f609a09ebe36450249b60d6abad04c53e61f7c
creating metadata file for /root/.cache/huggingface/transformers/a3ba5495a5249c5164e69fe430d3d126ad5d6427a55ee70274a812964bbc34f9.2f6fd7c2709e1ba46b4621a2d5f609a09ebe36450249b60d6abad04c53e61f7c
loading file https://huggingface.co/Rajan/NepaliBERT/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/a3ba5495a5249c5164e69fe430d3d126ad5d6427a55ee70274a812964bbc34f9.2f6fd7c2709e1ba46b4621a2d5f609a09ebe36450249b60d6abad04c53e61f7c
loading file https://huggingface.co/Rajan/NepaliBERT/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/Rajan/NepaliBERT/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/Rajan/NepaliBERT/resolve/main/special_tokens_map.json fro

In [ ]:
tokens1 = tokenizer.tokenize("यता काठमाडौंको चन्द्रागिरिमा पनि हिउँ खेल्न हिँडेका पाँच जना बालबालिका बेपत्ता भएका थिए।")

In [ ]:
tokens1

['यता',
 'काठमाडौको',
 'चन',
 '##दर',
 '##ागि',
 '##रिम',
 '##ा',
 'पनि',
 'हिउ',
 'खल',
 '##न',
 'हिड',
 '##का',
 'पाच',
 'जना',
 'बालबालिका',
 'ब',
 '##पत',
 '##ता',
 'भएका',
 'थिए',
 '।']

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!git clone https://huggingface.co/Rajan/NepaliBERT

fatal: destination path 'NepaliBERT' already exists and is not an empty directory.


In [ ]:
from transformers import BertTokenizer
vocab_file_dir = './NepaliBERT/' 
tokenizer_rajan = BertTokenizer.from_pretrained(vocab_file_dir,
                                     strip_accents=False,
                                      clean_text=False )

In [ ]:
tokens = tokenizer_rajan.tokenize("यता काठमाडौंको चन्द्रागिरिमा पनि हिउँ खेल्न हिँडेका पाँच जना बालबालिका बेपत्ता भएका थिए।")
tokens

['यता',
 'काठमाडौंको',
 'चन्द्रागिरि',
 '##मा',
 'पनि',
 'हिउँ',
 'खेल्न',
 'हिँडेका',
 'पाँच',
 'जना',
 'बालबालिका',
 'बेपत्ता',
 'भएका',
 'थिए',
 '।']

In [ ]:
dataset["train"][0]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12918
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1446
    })
})

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# tokenizer = AutoTokenizer.from_pretrained("Sakonii/de-berta-base-base-nepali", use_auth_token=True, cache_dir="/mnt/storage0/common/Sakonii") # xlm-roberta-base Sakonii/distilbert-base-nepali 
# # tokenizer = AutoTokenizer.from_pretrained("Shushant/nepaliBERT", model_max_length=512, cache_dir="/mnt/storage0/common/Sakonii")   # Shushant/nepaliBERT Rajan/NepaliBERT

def tokenize_function(example):
    return tokenizer_rajan(example["text"], truncation=True, padding=True, max_length = 510 )

dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_rajan)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12918
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1446
    })
})

In [ ]:
dataset["train"][0]

In [ ]:
# Make sure there are only 'input_ids', 'labels' and 'attention_mask' columns
remove_columns = ['text'] # 'token_type_ids'
dataset = dataset.map(remove_columns=remove_columns)  # (because the model expects the argument to be named labels)

dataset.set_format("torch")
dataset["train"].column_names

  0%|          | 0/12918 [00:00<?, ?ex/s]

  0%|          | 0/1446 [00:00<?, ?ex/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
dataset['train']['labels'][0]

tensor(0)

## Training


In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!git lfs install

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


In [ ]:
!git lfs pull

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Not in a git repository.


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Rajan/NepaliBERT", use_auth_token=True, num_labels=16)




Some weights of the model checkpoint at Rajan/NepaliBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rajan/NepaliBERT and are new

In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8 MB 5.6 MB/s 
     |████████████████████████████████| 144 kB 17.0 MB/s 
     |████████████████████████████████| 181 kB 22.7 MB/s 
     |████████████████████████████████| 63 kB 736 kB/s 


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/New",
    evaluation_strategy="epoch",
    logging_steps=5,
    learning_rate=3e-5,
    save_steps=50000, #
    weight_decay=0.01,
    fp16=True, 
    num_train_epochs=5,
    lr_scheduler_type='linear',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.0, 
    warmup_steps=0,
    seed=86,
    push_to_hub=True,
    report_to="wandb",    
    run_name="ClassificationOnrajanNepalibert",
    hub_model_id="CohleM/ClassificationOnNepaliBert", 
    save_strategy="epoch" 
)

PyTorch: setting up devices


In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(eval_pred):
#     metric1 = load_metric("accuracy")
#     metric2 = load_metric("f1")
    
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
#     f1 = metric2.compute(predictions=predictions, references=labels)["f1"]
#     return {"accuracy": accuracy, "f1": f1}

In [ ]:
# !sudo apt install git-lfs

In [ ]:
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer_rajan,
    compute_metrics=compute_metrics,
)

/New is already a clone of https://huggingface.co/CohleM/ClassificationOnNepaliBert. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


### deberta-base-nepali [ours]


In [ ]:
# lr=3e-5, bs=8, vram_load_titanv=94.73%
trainer.train()

***** Running training *****
  Num examples = 12918
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1615


Epoch,Training Loss,Validation Loss


In [ ]:
# lr=2e-5, bs=8, vram_load_titanv=94.73%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8075
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.346600,0.466475,0.860304
2,0.814200,0.446826,0.877593
3,0.358400,0.456466,0.884509
4,0.263800,0.558176,0.880360
5,0.108000,0.644176,0.870678


/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 8
Saving model checkpoint to /mnt/storage0/common/Sakonii/clf-16NepaliNews-train_/checkpoint-1615
Configuration saved in /mnt/storage0/common/Sakonii/clf-16NepaliNews-train_/checkpoint-1615/config.json
Model weights saved in /mnt/storage0/common/Sakonii/clf-16NepaliNews-train_/checkpoint-1615/pytorch_model.bin
tokenizer config file saved in /mnt/storage0/common/Sakonii/clf-16NepaliNews-train_/checkpoint-1615/tokenizer_config.json
Special tokens file saved in /mnt/storage0/common/Sakonii/cl

Upload file pytorch_model.bin:   0%|          | 32.0k/531M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at e65ed0dab556bf7e68ede7553c6bfe1294b22105 but expected 9e96016c0c4afca680513d3a6dd9d6b3cc80b4ff        
To https://huggingface.co/Sakonii/deberta-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/deberta-base-nepali-clf'

Error pushing update to the model card. Please read logs and retry.
$remote: error: cannot lock ref 'refs/heads/main': is at e65ed0dab556bf7e68ede7553c6bfe1294b22105 but expected 9e96016c0c4afca680513d3a6dd9d6b3cc80b4ff        
To https://huggingface.co/Sakonii/deberta-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/deberta-base-nepali-clf'



TrainOutput(global_step=8075, training_loss=0.32745903177065744, metrics={'train_runtime': 3047.9356, 'train_samples_per_second': 21.191, 'train_steps_per_second': 2.649, 'total_flos': 1.980527512928256e+16, 'train_loss': 0.32745903177065744, 'epoch': 5.0})

In [ ]:
wandb.finish()

### Sushant/nepaliBert


In [ ]:
# lr=2e-5, bs=12, vram_load_titanv=92.81%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 5385
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: sakonii (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.396300,0.575537,0.834716
2,0.346500,0.526764,0.850622
3,0.252400,0.562476,0.859613
4,0.385200,0.700212,0.838866
5,0.243800,0.752423,0.843707


/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 12
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-1077
Configuration saved in clf-16NepaliNews-train/checkpoint-1077/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-1077/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-1077/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-1077/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tok

Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at abe4835479ca65aa7ca092ab5cd8dbd3e3ba4845 but expected c63d6a278212d2f0f5fee48d0751d8c2d9149ba2        
To https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16'

Error pushing update to the model card. Please read logs and retry.
$remote: error: cannot lock ref 'refs/heads/main': is at abe4835479ca65aa7ca092ab5cd8dbd3e3ba4845 but expected c63d6a278212d2f0f5fee48d0751d8c2d9149ba2        
To https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16'



TrainOutput(global_step=5385, training_loss=0.3686351099275502, metrics={'train_runtime': 1432.7696, 'train_samples_per_second': 45.081, 'train_steps_per_second': 3.758, 'total_flos': 1.699647926059008e+16, 'train_loss': 0.3686351099275502, 'epoch': 5.0})

In [ ]:
Z# lr=3e-5, bs=8, vram_load_titanv=71.08%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8075
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.386600,0.625739,0.816044
2,0.757200,0.747594,0.816044
3,0.276100,0.794065,0.846473
4,0.352700,0.895345,0.847856
5,0.013700,1.003374,0.845781


/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 8
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-1615
Configuration saved in clf-16NepaliNews-train/checkpoint-1615/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-1615/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-1615/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-1615/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special toke

Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at 0a7067f7d9e5af65eec3fc5524909a593179a18e but expected 09e41e8f50a137fe301ef738ea5c9f968784f148        
To https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16'

Error pushing update to the model card. Please read logs and retry.
$remote: error: cannot lock ref 'refs/heads/main': is at 0a7067f7d9e5af65eec3fc5524909a593179a18e but expected 09e41e8f50a137fe301ef738ea5c9f968784f148        
To https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBERT-Sushant-clf-NepaliNews16'



TrainOutput(global_step=8075, training_loss=0.34104878414526074, metrics={'train_runtime': 1582.6562, 'train_samples_per_second': 40.811, 'train_steps_per_second': 5.102, 'total_flos': 1.699647926059008e+16, 'train_loss': 0.34104878414526074, 'epoch': 5.0})

### Rajan/NepaliBERT


In [ ]:
# lr=3e-5, bs=26, vram_load_titanv=99.08%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 26
  Total train batch size (w. parallel, distributed & accumulation) = 26
  Gradient Accumulation steps = 1
  Total optimization steps = 2485
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.638600,0.842308,0.731674
2,0.467000,0.713890,0.790456
3,0.321000,0.740518,0.799447
4,0.301700,0.714793,0.809820
5,0.172100,0.756029,0.807746


***** Running Evaluation *****
  Num examples = 1446
  Batch size = 26
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-497
Configuration saved in clf-16NepaliNews-train/checkpoint-497/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-497/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-497/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-497/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 26
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-994
Configuration saved in clf-16NepaliNews-train/checkpoint-994/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-994/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-994/tokenizer_config.json

Upload file pytorch_model.bin:   0%|          | 32.0k/313M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at a7658ef9563642b01d0ea45745020ffd36e21977 but expected 02f5c1d5c3dacefba19359c253e758bd707081bc        
To https://huggingface.co/Sakonii/nepaliBert-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBert-clf'

Error pushing update to the model card. Please read logs and retry.
$remote: error: cannot lock ref 'refs/heads/main': is at a7658ef9563642b01d0ea45745020ffd36e21977 but expected 02f5c1d5c3dacefba19359c253e758bd707081bc        
To https://huggingface.co/Sakonii/nepaliBert-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBert-clf'



TrainOutput(global_step=2485, training_loss=0.48791578015091436, metrics={'train_runtime': 735.0743, 'train_samples_per_second': 87.869, 'train_steps_per_second': 3.381, 'total_flos': 8558205474078720.0, 'train_loss': 0.48791578015091436, 'epoch': 5.0})

In [ ]:
# lr=2e-5, bs=26, vram_load_titanv=99.08%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 26
  Total train batch size (w. parallel, distributed & accumulation) = 26
  Gradient Accumulation steps = 1
  Total optimization steps = 2485
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.680200,0.809565,0.751037
2,0.531700,0.791739,0.777317
3,0.397400,0.721084,0.800830
4,0.331000,0.700526,0.801521
5,0.323200,0.724510,0.804288


***** Running Evaluation *****
  Num examples = 1446
  Batch size = 26
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-497
Configuration saved in clf-16NepaliNews-train/checkpoint-497/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-497/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-497/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-497/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/special_tokens_map.json
Several commits (2) will be pushed upstream.
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 26
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-994
Configuration saved in clf-16NepaliNews-train/checkpoint-994/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-994/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNe

Upload file pytorch_model.bin:   0%|          | 32.0k/313M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at e5d05ae3b634c8d7b38adb7a74d6828e596df469 but expected 511b188ff89812b98de8213a79e6dbc0c79f775f        
To https://huggingface.co/Sakonii/nepaliBert-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBert-clf'



OSError: remote: error: cannot lock ref 'refs/heads/main': is at e5d05ae3b634c8d7b38adb7a74d6828e596df469 but expected 511b188ff89812b98de8213a79e6dbc0c79f775f        
To https://huggingface.co/Sakonii/nepaliBert-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/nepaliBert-clf'


In [ ]:
# lr=5e-5, bs=24
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 2695
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.255200,1.220649,0.719917
2,0.124600,1.141809,0.776625


***** Running Evaluation *****
  Num examples = 1446
  Batch size = 24
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-539
Configuration saved in clf-16NepaliNews-train/checkpoint-539/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-539/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-539/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-539/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 24
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-1078
Configuration saved in clf-16NepaliNews-train/checkpoint-1078/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-1078/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-1078/tokenizer_config.

KeyboardInterrupt: 

In [ ]:
wandb.finish()

### distilbert-base-nepali [Ours]


In [ ]:
# lr=1e-5, bs=32, vram_load_titanv=99.59%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2020
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.698600,0.664462,0.827801
2,0.620700,0.558308,0.837483
3,0.419900,0.511091,0.854772
4,0.348800,0.481149,0.860304
5,0.387700,0.489439,0.856155


/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 32
Saving model checkpoint to clf-16NepaliNews-train_/checkpoint-404
Configuration saved in clf-16NepaliNews-train_/checkpoint-404/config.json
Model weights saved in clf-16NepaliNews-train_/checkpoint-404/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train_/checkpoint-404/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train_/checkpoint-404/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train_/tokenizer_config.json
Special to

Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at c2e48a44b215d44616ace7c2e88f35a30578c236 but expected d7f93e8c7c3b06ccb61b894d776c76f9c785a73e        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'



OSError: remote: error: cannot lock ref 'refs/heads/main': is at c2e48a44b215d44616ace7c2e88f35a30578c236 but expected d7f93e8c7c3b06ccb61b894d776c76f9c785a73e        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'


In [ ]:
# lr=2e-5, bs=32, vram_load_titanv=99.59%
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1616
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.597500,0.545496,0.843015
2,0.424700,0.456841,0.872752
3,0.290000,0.431343,0.883126
4,0.250100,0.432083,0.879668


***** Running Evaluation *****
  Num examples = 1446
  Batch size = 32
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-404
Configuration saved in clf-16NepaliNews-train/checkpoint-404/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-404/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-404/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-404/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/special_tokens_map.json
Several commits (2) will be pushed upstream.
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 32
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-808
Configuration saved in clf-16NepaliNews-train/checkpoint-808/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-808/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNe

Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at 39371db1bd3ce2d3331045a93babc33c8614ed91 but expected bc990c29900c85016238c6a7c3bcf69625cb8f12        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'

Error pushing update to the model card. Please read logs and retry.
$remote: error: cannot lock ref 'refs/heads/main': is at 39371db1bd3ce2d3331045a93babc33c8614ed91 but expected bc990c29900c85016238c6a7c3bcf69625cb8f12        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'



TrainOutput(global_step=1616, training_loss=0.5129271843240107, metrics={'train_runtime': 526.1828, 'train_samples_per_second': 98.202, 'train_steps_per_second': 3.071, 'total_flos': 6846514691586048.0, 'train_loss': 0.5129271843240107, 'epoch': 4.0})

In [ ]:
# lr=3e-5
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4040
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.547400,0.467114,0.867220
2,0.357700,0.456065,0.863071
3,0.281900,0.413905,0.876902
4,0.131300,0.528043,0.856846
5,0.102300,0.560290,0.860996
6,0.037200,0.630840,0.852006
7,0.011300,0.638768,0.863762
8,0.019000,0.694790,0.865145
9,0.004700,0.707877,0.864454
10,0.004800,0.716457,0.866528


***** Running Evaluation *****
  Num examples = 1446
  Batch size = 32
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-404
Configuration saved in clf-16NepaliNews-train/checkpoint-404/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-404/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-404/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-404/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/special_tokens_map.json
/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be requi

Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at a87e8c68f62f674cb8c990bf6408d4d1b9ad52ea but expected 7789a790c8a6bbbdcc649b30c520f991947ecc3c        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'



OSError: remote: error: cannot lock ref 'refs/heads/main': is at a87e8c68f62f674cb8c990bf6408d4d1b9ad52ea but expected 7789a790c8a6bbbdcc649b30c520f991947ecc3c        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'


In [ ]:
wandb.finish()

### xlm-roberta-base


In [ ]:
trainer.train()

/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12918
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16150
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.480900,0.830831,0.782849
2,1.205900,0.829219,0.760719
3,0.819400,0.619980,0.825726
4,0.686500,0.775610,0.807746
5,0.407300,0.659548,0.840249
6,0.693200,0.832789,0.817427
7,0.397200,0.854190,0.836791
8,0.187100,0.921899,0.839557
9,0.285900,1.061323,0.811895
10,0.428700,1.045193,0.817427


/mnt/storage0/utsavm/miniconda3/envs/NLMUT/lib/python3.9/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1446
  Batch size = 8
Saving model checkpoint to clf-16NepaliNews-train/checkpoint-1615
Configuration saved in clf-16NepaliNews-train/checkpoint-1615/config.json
Model weights saved in clf-16NepaliNews-train/checkpoint-1615/pytorch_model.bin
tokenizer config file saved in clf-16NepaliNews-train/checkpoint-1615/tokenizer_config.json
Special tokens file saved in clf-16NepaliNews-train/checkpoint-1615/special_tokens_map.json
tokenizer config file saved in clf-16NepaliNews-train/tokenizer_config.json
Special toke

Upload file pytorch_model.bin:   0%|          | 32.0k/1.04G [00:00<?, ?B/s]

remote: error: cannot lock ref 'refs/heads/main': is at eed81a0a0f907c3b76d8bfc897e30b762f4c09e4 but expected e930b42cc688ddb2507449c0b31f963f3e840e44        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'



OSError: remote: error: cannot lock ref 'refs/heads/main': is at eed81a0a0f907c3b76d8bfc897e30b762f4c09e4 but expected e930b42cc688ddb2507449c0b31f963f3e840e44        
To https://huggingface.co/Sakonii/distilbert-base-nepali-clf
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/Sakonii/distilbert-base-nepali-clf'


In [ ]:
wandb.finish()

eval/accuracy,▃▁▇▅█▆██▆▆
eval/loss,▄▄▁▃▂▄▅▆██
eval/runtime,▁▁▁▁▁▃▁▁█▁
eval/samples_per_second,█▇███▅██▁█
eval/steps_per_second,█▇███▅██▁█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▃▃▄▃▄▅▄▂▄▂▂▃▃▂▂▄▂▃▃▄▂▂▃▄▃▂▂▂▃▁▃▂▁▁▂▁▁
train/total_flos,▁
train/train_loss,▁


## Push to hub


In [ ]:
trainer.push_to_hub()

In [ ]:
wandb.finish()